# Исследование оттока клиентов из банка

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score 
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

data = pd.read_csv('/datasets/Churn.csv')

data.info()
display(data.shape)
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


(10000, 14)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [2]:
# Посчитаем среднее арифметическое и медианы по столбцу Tenure.
display(data['Tenure'].mean())
display(data['Tenure'].median())

# Среднее арифметическое и медиана для столбца 'Tenure' практически одинаковые.
median_days_employed = data['Tenure'].median()

# Заполним нулевые значения медианой.
data['Tenure'] = data['Tenure'].fillna(median_days_employed)

# Проверим, остались ли в таблице нулевые значения.
data.info()

4.997690023099769

5.0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
# Удалим ненужные для анализа столбцы
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace = True)

display(data.shape)
data.head()

(10000, 11)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
#Применим прямое кодирование к таблице
data_ohe = pd.get_dummies(data, drop_first=True)
display(data_ohe.shape)
display(data_ohe.head())

(10000, 12)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [5]:
# Выделим целевые признаки и условия
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)

# Разобьём данные на выборки
features_train_1, features_valid, target_train_1, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(features_train_1, target_train_1, test_size=0.25, random_state=12345)

# Проверим, правильно ли распределились данные
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

display (len(features_train) / (len(features_train) + len(features_valid) + len(features_test)))
display (len(features_valid) / (len(features_train) + len(features_valid) + len(features_test)))
display (len(features_test) / (len(features_train) + len(features_valid) + len(features_test)))


(6000, 11)

(2000, 11)

(2000, 11)

0.6

0.2

0.2

## Исследование задачи

In [6]:
# Обучим модель, измерим показатель F1
model = LogisticRegression(solver='liblinear', random_state = 12345) 
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.09896907216494845

In [7]:
# Обучим модель, измерим показатель F1
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5165876777251184

In [8]:
# Обучим модель, измерим показатель F1
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5295857988165681

## Борьба с дисбалансом

In [9]:
# Сделаем веса классов сбалансированными, посмотрим как изментися показатель F1
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.4451313755795981

In [10]:
# Сделаем веса классов сбалансированными, посмотрим как изментися показатель F1
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.514354066985646

In [11]:
# Сделаем веса классов сбалансированными, посмотрим как изментися показатель F1
model = RandomForestClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.482225656877898

**Вывод**

Наилучший показатель F1 без балансировки классов показала модель RandomForestClassifier. После балансировки классов модель LogisticRegression существенно увеличила показатель F1, а у остальных двух моделей данный показатель снизился.

In [12]:
# Произведем увеличение выборки, измерим показатель F1 для раздичных моделей
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 20)

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

model = RandomForestClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.3518747424804285

0.484394506866417

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5706666666666667

In [13]:
# Произведем уменьшение выборки, измерим показатель F1 для раздичных моделей
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)

model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

model = RandomForestClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.44271844660194176

0.4972200158856236

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5681211041852181

In [14]:
# Изменим порог для повышения показателя F1
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

display(roc_auc_score(target_valid, probabilities_one_valid))

for threshold in np.arange(0, 0.5, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    print(threshold, f1_score(target_valid, predicted_valid))

0.6711395906626906

0.0 0.3518747424804285
0.05 0.3546511627906977
0.1 0.3812129502963976
0.15000000000000002 0.39999999999999997
0.2 0.40498899486427
0.25 0.4075829383886256
0.30000000000000004 0.36494597839135656
0.35000000000000003 0.3130434782608696
0.4 0.23670668953687823
0.45 0.1511627906976744


In [15]:
# Изменим порог для повышения показателя F1
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

display(roc_auc_score(target_valid, probabilities_one_valid))

for threshold in np.arange(0, 0.5, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    print(threshold, f1_score(target_valid, predicted_valid))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8146332058403594

0.0 0.44703143189755523
0.05 0.44703143189755523
0.1 0.5340557275541795
0.15000000000000002 0.5340557275541795
0.2 0.573055028462998
0.25 0.573055028462998
0.30000000000000004 0.5957446808510638
0.35000000000000003 0.5957446808510638
0.4 0.5657894736842104
0.45 0.5657894736842104


In [16]:
# Определим оптимальное значение гиперпараметра n_estimators
for est in range (10, 70, 5):
    model = RandomForestClassifier(random_state=12345, n_estimators = est)
    model.fit(features_train, target_train)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    predicted_valid = probabilities_one_valid > 0.3
    print(est, f1_score(target_valid, predicted_valid))

10 0.5957446808510638
15 0.615546218487395
20 0.6237513873473918
25 0.6183368869936033
30 0.6262403528114664
35 0.6266094420600857
40 0.6162402669632926
45 0.6206896551724138
50 0.627062706270627
55 0.6282608695652173
60 0.6334440753045404
65 0.6295081967213114


**Выводы**

Увеличение выборки позволило вырастить показатель F1 для модели RandomForestClassifier, однако он остается на достаточно низком уровне, уменьшение выборки привело к снижению показателя F1 для указанной модели.

Переборкой в цикле значений порога удалось установить, что для модели RandomForestClassifier показатель F1 получает наибольшее значение при пороге, равном 0,3. Далее провден поиск оптимального значения гиперпараметра n_estimators, которое равно 60.

Для моделей LogisticRegression и RandomForestClassifier измерен показатель AUC-ROC. В первом случае данный показатель равен 0,67, метрика F1 также находится на достаточно низком уровне. При использовании модели RandomForestClassifier значение показателя AUC-ROC существенно выше - 0,81, при этом метрика F1 

## Тестирование модели

In [17]:
# Проведем тестирование на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=60)
model.fit(features_train, target_train)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.3
print(f1_score(target_test, predicted_test), roc_auc_score(target_test, predicted_test))

0.5945303210463734 0.7575426906515302


**Финальный вывод**

Нашей задачей в данном проекте было сформировать модель, которая поможет спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Для решения задачи нам была предоставлена статистическая таблица с различными параметрам и целевым признаком. 

В процессе задачи был загружен файл с данными, данные изучены, проведена их предобработка. 

Обучены три модели, однако метрики качества оказались на низком уровне, модели не пригодны для использования. Чтобы устранить данный недостаток предпринята попытка повысить значение метрик путем балансировки классов. Метрики улучшились, однако цель задачи не достигнута. Далее проведены увеличение и уменьшение выборки, что тоже не дало желаемый результат. 

Добиться требуемого значения метрики F1 удалось путем перебоки значений порогов для модели RandomForestClassifier.

Тестирование модели подтвердило, что желаемый результат достигнут. Модель можно использовать для прогнозирования ухода клиентов из банка.